# CNN Example 1
For this example, we have images of cars and flowers, which have been divided into training and testing sets, and we have to build a CNN that identifies whether an image is a car or a flower.

### Step 1: Import the numpy library and the necessary Keras libraries and classes

In [1]:
!pip install keras
!pip install tensorflow
!pip install --upgrade tensorflow


In [2]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np


from tensorflow import random

### Step 2: Now, set a seed and initiate the model with the `Sequential` class

In [3]:
#set a seed
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

### Step 3: Add the first layer of the CNN, set the input shape to (64, 64, 3), the dimension of each image, and set the activation function as a ReLU:

In [4]:


# Step 3: Add the first layer of the CNN
# Set the input shape to (64, 64, 3) for images with dimensions 64x64 and 3 color channels (RGB)
# Set the activation function as ReLU
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu'))


### Step 4: Now, add the pooling layer with the image size as 2x2

In [5]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

### Step 5: Flatten the output of the pooling layer by adding a flattening layer to the CNN model:

In [6]:
classifier.add(Flatten())

### Step 6: Add the first Dense layer of the MLP. 
Here, 128 is the output of the number of nodes. As a good practice, 128 is good to get started. activation is relu. As a good practice, the power of two is preferred

In [7]:
classifier.add(Dense(units=128, activation='relu'))

### Step 7: Add the output layer of the MLP.
This is a binary classification problem, so the size is 1 and the activation is `sigmoid`:

In [8]:
classifier.add(Dense(units=1, activation='sigmoid'))

### Step 8: Compile the network
Use an adam optimizer and compute the accuracy during the training process 

In [9]:
classifier.layers

In [10]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

### Step 9: Create training and test data generators. 
- Rescale the training and test images by `1/255` so that all the values are between `0` and `1`.
- Set these parameters for the training data generators only 
 - `shear_range=0.2`, `zoom_range=0.2`, and `horizontal_flip=True`
 
 - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [11]:
from keras.preprocessing.image import ImageDataGenerator
train_dataGen= ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True)
test_dataGen= ImageDataGenerator(rescale=1/255)

### Step 10: Create a training set from the training set folder.
'training_set' is the folder where our data has been placed. Our CNN model has an image size of `64x64`, so the same size should be passed here too. `batch_size` is the number of images in a single batch, which is `32`. `Class_mode` is set to binary since we are working on binary classifiers

In [12]:
# Step 10: Create a training set from the training set folder
training_set = train_dataGen.flow_from_directory(
    r'C:\Users\User\Desktop\car_flower\train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)




Found 1599 images belonging to 2 classes.


### Step 11: Repeat step 10 for the test set 
while setting the folder to the location of the test images, that is, 'test_set'

In [13]:
# Step 11 (continued): Create a test set from the test set folder
test_set = test_dataGen.flow_from_directory(
    r'C:\Users\User\Desktop\car_flower\test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

Found 398 images belonging to 2 classes.


### Step 12: Finally, fit the data. 
Set the `steps_per_epoch` to `STEP_SIZE_TRAIN` and the `validation_steps` to `STEP_SIZE_TEST`. 

Why do we need `steps_per_epoch` ?

Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the `steps_per_epoch` value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch.

In [14]:
from PIL import Image

def load_image(img_path):
    try:
        img = Image.open(img_path)
        return img
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

# Update the _get_batches_of_transformed_samples method in your data generator
class CustomImageDataGenerator(ImageDataGenerator):
    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = super()._get_batches_of_transformed_samples(index_array)
        
        # Load images with error handling
        batch_x_processed = []
        for img_path in self.filepaths[index_array]:
            img = load_image(img_path)
            if img is not None:
                batch_x_processed.append(img)
        
        return np.array(batch_x_processed), batch_x[1]

# Use your custom data generator
train_dataGen = CustomImageDataGenerator(rescale=1/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Create a training set from the training set folder
training_set = train_dataGen.flow_from_directory(
    r'C:\Users\User\Desktop\car_flower\train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# Continue with training...


Found 1599 images belonging to 2 classes.


In [15]:
import os
from PIL import Image

ignored_files = []

def find_invalid_images(folder_path):
    invalid_images = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                Image.open(file_path)
            except (IOError, OSError, Image.UnidentifiedImageError):
                invalid_images.append(file_path)
    return invalid_images

def remove_invalid_images(folder_path):
    global ignored_files  # global değişkeni kullanarak bu listeye erişim sağlıyoruz
    invalid_images = find_invalid_images(folder_path)
    for file_path in invalid_images:
        if file_path in ignored_files:
            continue  # Bu dosyayı ihmal et, devam et
        try:
            os.remove(file_path)
            print(f"Removing invalid image: {file_path}")
        except Exception as e:
            print(f"Error removing {file_path}: {e}")
            ignored_files.append(file_path)  # Hata aldığımız dosyayı ihmal et

# Test etmek için
remove_invalid_images(r'C:\Users\User\Desktop\car_flower\test\car_test')
remove_invalid_images(r'C:\Users\User\Desktop\car_flower\test\flower_test')
remove_invalid_images(r'C:\Users\User\Desktop\car_flower\train\car_train')
remove_invalid_images(r'C:\Users\User\Desktop\car_flower\train\flower_train')


In [16]:
# Step 12: Train the model using the generators
classifier.fit_generator(
    training_set,
    epochs=32,
    validation_data=test_set,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set))

C:\Users\User\AppData\Local\Temp\ipykernel_16820\2734127866.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/32


50/50 [==============================] - 11s 168ms/step - loss: 0.6788 - accuracy: 0.6585 - val_loss: 0.5187 - val_accuracy: 0.7538
Epoch 2/32
50/50 [==============================] - 8s 158ms/step - loss: 0.4793 - accuracy: 0.7780 - val_loss: 0.6500 - val_accuracy: 0.7161
Epoch 3/32
50/50 [==============================] - 8s 161ms/step - loss: 0.4414 - accuracy: 0.7955 - val_loss: 0.6631 - val_accuracy: 0.7261
Epoch 4/32
50/50 [==============================] - 8s 158ms/step - loss: 0.4148 - accuracy: 0.8093 - val_loss: 0.5258 - val_accuracy: 0.7513
Epoch 5/32
50/50 [==============================] - 8s 158ms/step - loss: 0.3997 - accuracy: 0.8205 - val_loss: 0.5417 - val_accuracy: 0.7538
Epoch 6/32
50/50 [==============================] - 8s 163ms/step - loss: 0.3673 - accuracy: 0.8412 - val_loss: 0.4914 - val_accuracy: 0.7764
Epoch 7/32
50/50 [==============================] - 8s 162ms/step - loss: 0.3686 - accuracy: 0.8393 - val_loss: 0.5096 - val_accuracy: 0.7814
Epo